# Importing numpy, sep, matplotlib, and astropy

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Reading an example FITS file and displaying it

In [ ]:
data = fits.getdata('image.fits')

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('fig1.png')

# Getting the background data of the image (mean and noise)

In [ ]:
bkg = sep.Background(data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

# Evaluating background as 2-d array like original image

In [ ]:
bkg_image = bkg.back()

# Showing the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('fig2.png')

# Evaluating background noise as 2-d array like original image

In [ ]:
bkg_rms = bkg.rms()

# Showing the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('fig3.png')

# Subtracting the background

In [ ]:
data_sub = data - bkg

# Running object detection now that background is gone

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# How many objects there are

In [ ]:
len(objects)

# 'Circling' (ellipsing) the objects in the new background subtracted image

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('fig4.png')

# Showing the available fields

In [ ]:
objects.dtype.names

# Performing circular aperture photometry at locations of each object

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

# Showing first 10 object results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))